# 실습설명
화장품 이커머스 데이터 분석을 통해, 고객의 이탈율을 낮추고 구매 전환율을 높이는 것이 목표

# 데이터 설명

- event_time : 이벤트가 발생한 시간
- event_type : 이벤트 유형(구매)
- product_id : 제품ID
- category_id : 제품의 카테고리 ID
- category_code : 제품의 카테고리 코드 (의미있는 카테고리의 경우 존재)
- brand : 브랜드 이름 (소문자, 생략될 수 있음)
- price : 제품의 가격
- user_id : 사용자 ID
- user_session : 사용자 세션 ID(임시, 세션마다 변경됨)

In [1]:
import pandas as pd
df  = pd.read_csv("/Users/jangminseul/Desktop/ecommerce/data/raw/cosmetic.csv")
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


In [12]:
print(df.shape) # 데이터 개수
print(df.info()) # 컬럼별 타입

(3533286, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3533286 entries, 0 to 3533285
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 242.6+ MB
None


In [16]:
df['event_time'] = pd.to_datetime(df['event_time']) # 데이터 타입 변환

In [17]:
df['event_time'].dtype

datetime64[ns, UTC]

In [18]:
df.duplicated().sum() # 중복데이터 확인

183860

중복 데이터 원인
- 같은 행동이 동시에 여러번 기록
- 초 단위 timestamp
- 세션 로그 특성

In [20]:
df.duplicated().sum()/len(df) # 중복 데이터 비율 확인

0.052036546149957855

In [21]:
df[df.duplicated()].head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
37,2019-12-01 00:01:52+00:00,cart,5700046,1487580009286598681,NaN,runail,0.40,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
55,2019-12-01 00:02:49+00:00,cart,5693501,1487580009445982239,NaN,NaN,0.60,579751441,8de492d7-0937-47ae-be2c-a7615aec2b0d
61,2019-12-01 00:03:08+00:00,remove_from_cart,5859482,1487580005671109489,NaN,masura,1.60,561162056,39cf2227-03ed-421e-9615-7814b9b3c5e6
62,2019-12-01 00:03:08+00:00,remove_from_cart,5839671,1487580005671109489,NaN,masura,2.19,561162056,39cf2227-03ed-421e-9615-7814b9b3c5e6
66,2019-12-01 00:03:10+00:00,remove_from_cart,5859482,1487580005671109489,NaN,masura,1.60,561162056,39cf2227-03ed-421e-9615-7814b9b3c5e6


In [22]:
df.duplicated(subset=['user_id','event_time','product_id','event_type']).sum()

185220

In [23]:
df.drop_duplicates(subset=['user_id','event_time','product_id','event_type'])

# 같은 사용자가, 같은 시점에, 같은 상품에 대해 같은 행동을 여러번 기록 한 경우로 바라보고 분석하기 위함

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00+00:00,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00+00:00,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02+00:00,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05+00:00,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07+00:00,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5
...,...,...,...,...,...,...,...,...,...
3533281,2019-12-31 23:59:39+00:00,view,5683350,1487580005671109489,NaN,masura,2.84,536812729,e4a2d47c-a956-4c46-8176-745f52ea664b
3533282,2019-12-31 23:59:46+00:00,view,5888097,1487580013388628160,NaN,shik,179.05,503658154,2dde9867-9e71-4a64-880d-aa68b66aae6d
3533283,2019-12-31 23:59:51+00:00,view,59975,1487580012096782476,NaN,NaN,7.14,595414541,4c6d80bb-5dd3-4fbb-b592-187b51db2753
3533284,2019-12-31 23:59:52+00:00,view,5775982,1783999063314661546,NaN,NaN,11.90,397780878,7e8a2b85-153a-44eb-a71f-b748fde14fcc


In [14]:
print(df.isnull().sum()) # 결측치 확인
(df.isnull().sum()/len(df))*100  # 결측치 비율 확인

event_time             0
event_type             0
product_id             0
category_id            0
category_code    3474821
brand            1510289
price                  0
user_id                0
user_session         779
dtype: int64


event_time        0.000000
event_type        0.000000
product_id        0.000000
category_id       0.000000
category_code    98.345308
brand            42.744601
price             0.000000
user_id           0.000000
user_session      0.022047
dtype: float64

In [11]:
df['event_type'].value_counts() 
# 퍼널 구조 view -> cart -> purchase
# 중간 이탈 remove_from_cart

event_type
view                1728331
cart                 927124
remove_from_cart     664655
purchase             213176
Name: count, dtype: int64

In [15]:
# 단계별 전환율
view = 1728331
cart = 927124
remove_from_cart = 664655
purchase = 213176

view_to_cart = cart / view
cart_to_purchase = purchase / cart
view_to_purchase = purchase / view

view_to_cart,cart_to_purchase,view_to_purchase

(0.5364273394390311, 0.22993256565464815, 0.12334211444451323)